# Walmart Sales Forecasting Project

## Sales Forecasting

Sales Forecasting is a very important area in field of business management. It helps the store retailers to maintain there stocks according to demand they are expecting, thus maximizing their profit and minimise burden of management of product stocks. It is similar to weather forecasting as both types of forecasting rely on science and historical data.
While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses.
Forecasting can drive sales by processing just-in-time orders efficiently.

## Problem Statement

We have Sales data for Walmart Stores in 3 states(California, Texas, Wisconsin) for 3 categoies of data (HOBBIES, FOOD, HOSEHOLD) from year 2011 to 2016. We want to use this data to predict sales for next 28 days using ML techniques.

## Dataset


We have taken dataset from kaggle Competition named "M5 Forecasting - Accuracy Estimate the unit sales of Walmart retail goods"

We have been given sales of product for 1913 days amd we need to predict sales of next 28 days for each product.

We have been given several Dataframes Like:-


* calendar.csv - Contains information about the dates on which the products are sold.
* sell_prices.csv - Contains information about the price of the products sold per store and date.
* sales_train_validation.csv - Contains the historical daily unit sales data per product and store d_1 - d_1913.
* sales_train_evaluation.csv - Includes sales d_1 - d_1941.

## Metric Used

* This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE).
* We have used WRMSSE instead of Simple RMSE(Root Mean Squared Error) because in this dataset there are lot of zero sales so even if our model predicts most sales near to zero our model will give good performance, WRMSSE takes this in account.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from lightgbm import Dataset,train,plot_importance
from sklearn import preprocessing, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, RidgeCV
import gc
import os
import random
from itertools import cycle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
random.seed(42)
np.random.seed(42)

In [3]:
def reduce_mem_usage(df, verbose=True):
    
    '''
    reduce the memory usage of the dataframe by downcasting 
    the int and float to avoid the memory error
    '''
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
def read_data(evaluation):
    
    '''
    read data and reduce memory
    '''
    
    print('Reading files...')
    
    calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    if evaluation:
        sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
        print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    else:
        sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
        print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
          
    submission = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sample_submission.csv')
    submission = reduce_mem_usage(submission)
    
    return calendar, sell_prices, sales_train_validation, submission

In [5]:
def melt_and_merge_for_eda(calendar, sell_prices, sales_train_validation, submission, nrows = 30000000):
    
    sales_train_validation = pd.melt(sales_train_validation,
                                     id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                     var_name = 'day',
                                     value_name = 'demand')
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test2 = submission[submission['id'].isin(test2_rows)]

    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

    test2 = test2.merge(product, how = 'left', on = 'id')
    test2 = pd.melt(test2, 
                    id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    var_name = 'day', 
                    value_name = 'demand')

    sales_train_validation['part'] = 'train'
    test2['part'] = 'test'
    
    data = pd.concat([sales_train_validation,test2], axis = 0)
    del sales_train_validation,test2
    gc.collect()
    
    data.reset_index(drop=True,inplace=True)
    data = data.loc[30000000:]
    #calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
    data.drop(['d', 'day'], inplace = True, axis = 1)
    del calendar,product
    gc.collect()
    
    data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    del sell_prices
    gc.collect()
    
    print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    
    return data

In [6]:
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 30000000):
    
    sales_train_validation = pd.melt(sales_train_validation,
                                     id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                                     var_name = 'day',
                                     value_name = 'demand')
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test2 = submission[submission['id'].isin(test2_rows)]

    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

    test2 = test2.merge(product, how = 'left', on = 'id')
    test2 = pd.melt(test2, 
                    id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                    var_name = 'day', 
                    value_name = 'demand')

    sales_train_validation['part'] = 'train'
    test2['part'] = 'test'
    
    data = pd.concat([sales_train_validation,test2], axis = 0)
    del sales_train_validation,test2
    gc.collect()
    
    data.reset_index(drop=True,inplace=True)
    data = data.loc[30000000:]
    calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
    data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
    data.drop(['d', 'day'], inplace = True, axis = 1)
    
    data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
    del calendar,sell_prices,product
    gc.collect()
    
    print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    
    return data

In [7]:
def transform(data):
    
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
                
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
        
    print('fillna and encoded')
    
    return data

In [8]:
def features_engineering(data):
    
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data = reduce_mem_usage(data)
    
    data['rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())

    data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1'] = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365'] = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data['rolling_price_std_t7'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['week'] = data['date'].dt.week
    data['day'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    data['isweekend'] = data['dayofweek'].apply(lambda x: 1 if x==5 or x==6 else 0)
    
    data['revenue'] = data['demand'] * data['sell_price']
    data['lag_revenue_t1'] = data.groupby(['id'])['revenue'].transform(lambda x: x.shift(28))
    data['rolling_revenue_std_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).std())
    data['rolling_revenue_mean_t28'] = data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).mean())
    data.drop(['revenue'],axis=1,inplace=True)
    data = reduce_mem_usage(data)
    
    return data

In [9]:
def split_data(data):
    x_train = data[data['part'] == 'train']
    y_train = x_train['demand']
    x_val = data[(data['date'] > '2016-04-24') & (data['part'] == 'train')]
    y_val = x_val['demand']
    test = data[data['date'] > '2016-04-24']
    test.loc[test['part']=='train','id'] = test.loc[test['part']=='train','id'].str.replace('_evaluation','_validation')
    x_train.drop(['demand','part',],inplace=True,axis=1)
    x_val.drop(['demand','part',],inplace=True,axis=1)
    test.drop(['demand','part',],inplace=True,axis=1)
    
    del data
    gc.collect()
    
    return x_train, y_train, x_val, y_val, test

In [10]:
def dataset(category,category2):
    
    tindex = x_train[(x_train['dept_id']==category) & (x_train['store_id']==category2)].index.values
    vindex = x_val[(x_val['dept_id']==category)&(x_val['store_id']==category2)].index.values
    
    x_t,x_v,y_t,y_v,t =  x_train[(x_train['dept_id']==category) & (x_train['store_id']==category2)],x_val[(x_val['dept_id']==category)&(x_val['store_id']==category2)],y_train.loc[tindex],y_val.loc[vindex],test[(test['dept_id']==category) &(test['store_id']==category2)]
    
    x_train.drop(tindex,axis=0,inplace=True)
    x_val.drop(vindex,axis=0,inplace=True)
    test.drop(test[(test['dept_id']==category) &(test['store_id']==category2)].index.values,axis=0,inplace=True)
    
    return x_t,x_v,y_t,y_v,t

In [11]:
calendar, sell_prices, sales_train_validation, submission = read_data(True)

In [12]:
print("Shape of calender.csv", calendar.shape)
print("Shape of sell_price.csv", sell_prices.shape)
print("Shape of sales_train_validation.csv", sales_train_validation.shape)
print("Shape of submission.csv",submission.shape)

In [13]:
calendar.head()

In [14]:
sell_prices.head()

In [15]:
sales_train_validation.head()

In [16]:
submission.head()

In [17]:
data = melt_and_merge_for_eda(calendar, sell_prices, sales_train_validation,
                      submission, nrows = 30000000)

In [18]:
nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for feature in nan_features:
    data[feature].fillna('unknown', inplace = True)

In [19]:
data.head()

In [20]:
data.columns.values

In [21]:
data.info()

In [22]:
data['store_id'].unique()

In [23]:
data['state_id'].unique()

In [24]:
(data['date'].min(), data['date'].max())

In [25]:
sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales_train_validation = reduce_mem_usage(sales_train_validation)

In [26]:
df=data.groupby('cat_id').mean()
df.reset_index(level=0,inplace=True)
plt.figure(figsize=(20,10))
plt.subplot(121)
sns.barplot(x='cat_id',y='demand',data=df)
plt.title("Bar-Graph for Avg Sales According to each categories")

plt.subplot(122)
df=data.groupby('cat_id').sum()
df.reset_index(level=0,inplace=True)
df['perc']=df['demand']/sum(df['demand'].values)*100
plt.pie(df['perc'].values,labels=df['cat_id'].values,shadow=True,autopct='%1.1f%%')
plt.title("Pie Chart showing total sales for each categories")

plt.show()

Total sales for FOODS is the most.

In [27]:
sales_train_validation['total_sales'] = sales_train_validation.sum(axis=1)
sns.catplot(x='cat_id', y='total_sales',
           hue='state_id',
           data=sales_train_validation, kind='bar',
           height=8, aspect=1)
plt.title('Total Sales on Categories by State')
plt.show()

In [28]:
df=data.groupby('state_id').mean()
df.reset_index(level=0,inplace=True)
plt.figure(figsize=(20,10))
plt.subplot(121)
sns.barplot(x='state_id',y='demand',data=df)
plt.title("Bar-Graph for Avg Sales According to each state")

df=data.groupby('state_id').sum()
df.reset_index(level=0,inplace=True)
df['perc']=df['demand']/sum(df['demand'].values)*100
plt.subplot(122)
plt.pie(df['perc'].values,labels=df['state_id'].values,shadow=True,autopct='%1.1f%%')
plt.title("Pie Chart Total showing sales for each state")

plt.show()

California has the most sales. The reason might be because California has the most population compares to the other two states.

In [29]:
df=data.groupby('store_id').mean()
df.reset_index(level=0,inplace=True)
plt.figure(figsize=(20,10))
plt.subplot(121)
sns.barplot(x='store_id',y='demand',data=df)
plt.title("Bar-Graph for  AVG Sales According to each store")
df=data.groupby('store_id').sum()
df.reset_index(level=0,inplace=True)
df['perc']=df['demand']/sum(df['demand'].values)*100
plt.subplot(122)
plt.pie(df['perc'].values,labels=df['store_id'].values,shadow=True,autopct='%1.1f%%')
plt.title("Pie Chart showing total sales for each store")

plt.show()

At store CA_3, total sales is the most and it might be a cause of highest state sales.
Store WI_2 is also interesting, it has a lot of sales compares to another stores in the state.

In [30]:
sns.catplot(x='store_id', y='total_sales',
           hue='cat_id',
           data=sales_train_validation, kind='bar',
           height=8, aspect=1)
plt.title('Total Sales by Store')
plt.show()

Usually HOUSEHOLD product sales more than HOBBIES, but some store has HOBBIES product sales more than HOUSEHOLD.

In [31]:
df=data.groupby('dept_id').mean()
df.reset_index(level=0,inplace=True)
plt.figure(figsize=(20,10))
plt.subplot(121)
sns.barplot(x='dept_id',y='demand',data=df)
plt.title("Bar-Graph for  AVG Sales According to each department")
df=data.groupby('dept_id').sum()
df.reset_index(level=0,inplace=True)
df['perc']=df['demand']/sum(df['demand'].values)*100
plt.subplot(122)
plt.pie(df['perc'].values,labels=df['dept_id'].values,shadow=True,autopct='%1.1f%%')
plt.title("Pie Chart showing total sales for each department")

plt.show()

Almost 50% of the sales are done by FOODS_3 department.

In [49]:
df=data.groupby('month').mean()
df.reset_index(level=0,inplace=True)
plt.figure(figsize=(20,10))
plt.subplot(121)
sns.lineplot(x='month',y='demand',data=df, marker="o")
plt.yticks(np.arange(0.7, 1.5, 0.1))
plt.title("Line plot for AVG Sales According to each month")

df=data.groupby('month').sum()
df.reset_index(level=0,inplace=True)
df['perc']=df['demand']/sum(df['demand'].values)*100
plt.subplot(122)
plt.pie(df['perc'].values,labels=df['month'].values,shadow=True,autopct='%1.1f%%')
plt.title("Pie Chart showing Total sales for each month")

plt.show()

There are some seasonality in month sales data.

In [33]:
hobbies_state = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'HOBBIES')].groupby(['state_id']).mean().T
hobbies_state = hobbies_state.rename({'CA': 'HOBBIES_CA', 'TX': 'HOBBIES_TX', 'WI': 'HOBBIES_WI'}, axis=1)
household_state = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'HOUSEHOLD')].groupby(['state_id']).mean().T
household_state = household_state.rename({'CA': 'HOUSEHOLD_CA', 'TX': 'HOUSEHOLD_TX', 'WI': 'HOUSEHOLD_WI'}, axis=1)
foods_state = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'FOODS')].groupby(['state_id']).mean().T
foods_state = foods_state.rename({'CA': 'FOODS_CA', 'TX': 'FOODS_TX', 'WI': 'FOODS_WI'}, axis=1)
nine_example = pd.concat([hobbies_state, household_state, foods_state], axis=1)
nine_example = nine_example.drop('total_sales')

In [34]:
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

fig, axs = plt.subplots(3, 3, figsize=(15,15))
axs = axs.flatten()
ax_idx = 0
for item in nine_example.columns:
    nine_example[item].plot(title=item, color=next(color_cycle), ax=axs[ax_idx])
    ax_idx += 1
plt.tight_layout()
plt.show()

In [35]:
event_date = calendar.loc[calendar['event_name_1'].isin(calendar.event_name_1.unique()[1:])].d
FOODS_event = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'FOODS')].groupby(['state_id']).mean().T.reset_index()
FOODS_event = FOODS_event.loc[FOODS_event['index'].isin(event_date)]
plt.figure(figsize=(15, 10))
plt.subplot(3,1,1)
nine_example['FOODS_CA'].plot(title='FOODS_CA', color=next(color_cycle))
plt.scatter(FOODS_event.reset_index().level_0, FOODS_event['CA'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,2)
nine_example['FOODS_TX'].plot(title='FOODS_TX', color=next(color_cycle))
plt.scatter(FOODS_event.reset_index().level_0, FOODS_event['TX'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,3)
nine_example['FOODS_WI'].plot(title='FOODS_WI', color=next(color_cycle))
plt.scatter(FOODS_event.reset_index().level_0, FOODS_event['WI'],color=next(color_cycle), zorder=10)
plt.tight_layout()
plt.show()

In [36]:
HOBBIES_event = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'HOBBIES')].groupby(['state_id']).mean().T.reset_index()
HOBBIES_event = HOBBIES_event.loc[HOBBIES_event['index'].isin(event_date)]
plt.figure(figsize=(15, 10))
plt.subplot(3,1,1)
nine_example['HOBBIES_CA'].plot(title='HOBBIES_CA', color=next(color_cycle))
plt.scatter(HOBBIES_event.reset_index().level_0, HOBBIES_event['CA'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,2)
nine_example['HOBBIES_TX'].plot(title='HOBBIES_TX', color=next(color_cycle))
plt.scatter(HOBBIES_event.reset_index().level_0, HOBBIES_event['TX'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,3)
nine_example['HOBBIES_WI'].plot(title='HOBBIES_WI', color=next(color_cycle))
plt.scatter(HOBBIES_event.reset_index().level_0, HOBBIES_event['WI'],color=next(color_cycle), zorder=10)
plt.tight_layout()
plt.show()

In [37]:
HOUSEHOLD_event = sales_train_validation.loc[(sales_train_validation['cat_id'] == 'HOUSEHOLD')].groupby(['state_id']).mean().T.reset_index()
HOUSEHOLD_event = HOUSEHOLD_event.loc[HOUSEHOLD_event['index'].isin(event_date)]
plt.figure(figsize=(15, 10))
plt.subplot(3,1,1)
nine_example['HOUSEHOLD_CA'].plot(title='HOUSEHOLD_CA', color=next(color_cycle))
plt.scatter(HOUSEHOLD_event.reset_index().level_0, HOUSEHOLD_event['CA'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,2)
nine_example['HOUSEHOLD_TX'].plot(title='HOUSEHOLD_TX', color=next(color_cycle))
plt.scatter(HOUSEHOLD_event.reset_index().level_0, HOUSEHOLD_event['TX'],color=next(color_cycle), zorder=10)
plt.subplot(3,1,3)
nine_example['HOUSEHOLD_WI'].plot(title='HOUSEHOLD_WI', color=next(color_cycle))
plt.scatter(HOUSEHOLD_event.reset_index().level_0, HOUSEHOLD_event['WI'],color=next(color_cycle), zorder=10)
plt.tight_layout()
plt.show()

In [38]:
df=data.groupby(['year','month','state_id']).sum()
df.reset_index(level=[0,1,2],inplace=True)
plt.figure(figsize=(20,25))
plt.subplot(411)
plt.plot(df[df['state_id']=="CA"]['demand'].values,label="CA")
plt.plot(df[df['state_id']=="TX"]['demand'].values,label="TX")
plt.plot(df[df['state_id']=="WI"]['demand'].values,label="WI")
plt.xlabel('Months in incresing order of years')
plt.ylabel('sales')
plt.title("Sales of each state in Increasing order of months for each year")
plt.legend()

plt.subplot(412)
sns.distplot(df[df['state_id']=="CA"]['demand'].values)
plt.title("Distribution of sales for each month in various years for states of CA")

plt.subplot(413)
sns.distplot(df[df['state_id']=="TX"]['demand'].values)
plt.title("Distribution of sales for each month in various years for state of TX")

plt.subplot(414)
sns.distplot(df[df['state_id']=="WI"]['demand'].values)
plt.title("Distribution of sales for each month in various years for state of WI")

plt.show()

In [39]:
df=data.groupby(['year','month','cat_id']).sum()
df.reset_index(level=[0,1,2],inplace=True)
plt.figure(figsize=(20,25))
plt.subplot(411)
plt.plot(df[df['cat_id']=="HOBBIES"]['demand'].values,label="HOOBIES")
plt.plot(df[df['cat_id']=="FOODS"]['demand'].values,label="FOODS")
plt.plot(df[df['cat_id']=="HOUSEHOLD"]['demand'].values,label="HOUSEHOLD")
plt.xlabel('Months in incresing order of years')
plt.ylabel('sales')
plt.title("Sales of each category in Increasing order of months for each year")
plt.legend()
plt.subplot(412)
sns.distplot(df[df['cat_id']=="HOBBIES"]['demand'].values)
plt.title("Distribution of sales for each month in various years HOBBIES Category")

plt.subplot(413)
sns.distplot(df[df['cat_id']=="FOODS"]['demand'].values)
plt.title("Distribution of sales for each month in various years FOODS Category")

plt.subplot(414)
sns.distplot(df[df['cat_id']=="HOUSEHOLD"]['demand'].values)
plt.title("Distribution of sales for each month in various years HOUSEHOLD Category")

plt.show()

## Linear Regression

In [10]:
# Function to merge datasets returning only one dataframe.
def reshape_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 30000000, merge = False):
    
    # Reshaping sales data using melt.
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print ('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_trian_validation = reduce_mem_usage(sales_train_validation)
    
    # Preparing test and validation records.
    test_rows = [row for row in submission['id'] if 'validation' in row]
    val_rows = [row for row in submission['id'] if 'evaluation' in row]
    
    test = submission[submission ['id']. isin(test_rows)]
    val = submission[submission ['id']. isin(val_rows)]
    
    # Renaming the columns.
    test.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921',
                    'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930',
                    'd_1931', 'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939',
                    'd_1940', 'd_1941']
    val.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949',
                   'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958',
                   'd_1959', 'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967',
                   'd_1968', 'd_1969']
    
    # Getting only product data and removing duplicate records.
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]. drop_duplicates ()
    
    # Merge with the product table.
    test = test.merge(product, how = 'left', on = 'id')
    val = val.merge(product, how = 'left', on = 'id')
    
    # Reshaping test and validation data.
    test = pd.melt(test, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    val = pd.melt(val, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    # Creating a new column to define training, test and validation data.
    sales_train_validation['part'] = 'train'
    test['part'] = 'test'
    val['part'] = 'val'
    
    # Creating a single dataset with the addition of all training, validation and test records.
    data = pd.concat([sales_train_validation, test, val], axis = 0)
    
    # Removing previous datasets.
    del sales_train_validation, test, val
    
    # Selecting only a few records for training.
    data = data.loc[nrows:]
    
    # Removing validation data.
    data = data[data ['part']!= 'val']
    
    # Performing the merge with calendar and price.
    if merge:
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day', 'weekday'], inplace = True, axis = 1)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Final dataset for training has {} rows and {} columns'.format(data.shape [0], data.shape [1]))
        del calendar,sell_prices,product
        gc.collect()
    else:
        pass
    
    return data

# Function for handling missing values and transformation of categorical/numeric features
def transform2(data):
    
    # Performing treatment on missing values for the categorical features.
    nan_features_cat = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in nan_features_cat:
        data[feature] .fillna('unknown', inplace = True)
    
    # Performing treatment on missing values for the sell_price feature.
    data['sell_price']. fillna(0, inplace = True)
        
    # Turning features categories into numbers to make predictions.
    encoder = preprocessing.LabelEncoder ()
    data['id_encode'] = encoder.fit_transform(data ['id'])
    
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
           'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        encoder = preprocessing.LabelEncoder()
        data[feature] = encoder.fit_transform(data[feature])
    
    return data

In [6]:
def plotModelResults(model, X_train, X_test, plot_intervals=False, plot_anomalies=False):
    """
        Plots modeled vs fact values, prediction intervals and anomalies
    
    """
    
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(15, 7))
    plt.plot(y_test.values, label="actual", linewidth=2.0)
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, 
                                    scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale * deviation)
        upper = prediction + (mae + scale * deviation)
        
        plt.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        plt.plot(upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_test))
            anomalies[y_test<lower] = y_test[y_test<lower]
            anomalies[y_test>upper] = y_test[y_test>upper]
            plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    error = rmse(y_test, prediction)
    plt.title("RMSE: {0:.2f}".format(error))
    plt.legend(loc="best")
    plt.tight_layout()
    plt.grid(True);
    
def plotCoefficients(model):
    """
        Plots sorted coefficient values of the model
    """
    
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');

In [7]:
def prepare_data_lr(data, lag_start, lag_end, test_size, target_encoding=False):

    # lags of series
    for i in range(7, 29):
        data["lag_{}".format(i)] = data['demand'].shift(i)

    # datetime features
    data.index = pd.to_datetime(data.index)
    data["day"] = data.index.day
    data["weekday"] = data.index.weekday
    data['is_weekend'] = data.weekday.isin([5,6])*1

    if target_encoding:
        # calculate averages on train set only
        test_index = int(len(data.dropna())*(1-test_size))
        data['weekday_average'] = list(map(code_mean(data[:test_index], 'weekday', "demand").get, data['weekday']))
        data["day_average"] = list(map(code_mean(data[:test_index], 'day', "demand").get, data['day']))

        # frop encoded variables 
        data.drop(["day", "weekday"], axis=1, inplace=True)
    
    # train-test split
    y = data.dropna()['demand']
    X = data.dropna().drop(['demand'], axis=1)

    X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=test_size)

    return X_train, X_test, y_train, y_test

In [8]:
def code_mean(data, cat_feature, real_feature):
    """
    Returns a dictionary where keys are unique categories of the cat_feature,
    and values are means over real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test  = X.iloc[test_index:]
    y_test  = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [9]:
calendar, sell_prices, sales_train_validation, submission = read_data(False)

In [11]:
# Reshaping and merging datasets.
data = reshape_and_merge(calendar, sell_prices, sales_train_validation, submission, merge = True)

# Calling up the data transformation functions.
data = transform2(data)

# Viewing the final dataset header.
data.head()

# Clearing data from memory.
gc.collect()

In [12]:
# Selecting only 1 item for testing: FOODS_3_634_WI_2.
df = data [(data ['date'] <= '2016-04-24') & (data ['id'] == 'FOODS_3_634_WI_2_validation') & (data ['demand'] > 0) & (data['demand'] <= 15)]

# Selecting only a few columns for analysis and training.
df = df[['date', 'demand', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'snap_WI', 'sell_price']]

# Transforming the date as index.
df = df.set_index('date')

In [13]:
tscv = TimeSeriesSplit (n_splits = 5)

X_train, X_test, y_train, y_test = prepare_data_lr(df, lag_start=1, 
                                               lag_end=29, test_size=0.1, 
                                               target_encoding=True)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelResults(lr, X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True, plot_anomalies=True)
plotCoefficients(lr)

In [14]:
ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)

plotModelResults(ridge, 
                 X_train=X_train_scaled, 
                 X_test=X_test_scaled, 
                 plot_intervals=True, 
                 plot_anomalies=True)
plotCoefficients(ridge)

## LGBM Regressor

In [ ]:
calendar, sell_prices, sales_train_validation, submission = read_data(True)

In [ ]:
data = melt_and_merge(calendar, sell_prices, sales_train_validation,
                      submission, nrows = 30000000)

In [13]:
data = transform(data)

In [14]:
data = features_engineering(data)

In [15]:
x_train, y_train, x_val, y_val, test = split_data(data)

In [16]:
features = ['item_id', 'cat_id', 'state_id', 'year', 'month', 'week', 'day', 'dayofweek', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
            'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t30', 'rolling_mean_t90', 
            'rolling_mean_t180', 'rolling_std_t30', 'price_change_t1', 'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t30', 'rolling_skew_t30', 'rolling_kurt_t30',
            'isweekend','lag_revenue_t1','rolling_revenue_std_t28','rolling_revenue_mean_t28']
category = x_train['dept_id'].unique()
category2 = x_train['store_id'].unique()


from lightgbm import Dataset,train,plot_importance
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1}

TEST = None
#i=0
#feature_importances = pd.DataFrame()
#feature_importances['feature'] = features

for i in category:
    for j in category2:
        x_t,x_v,y_t,y_v,t = dataset(i,j)
        train_set = Dataset(x_t[features], y_t)
        val_set = Dataset(x_v[features], y_v)
        del x_t, y_t
        gc.collect()

        model = train(params, train_set, num_boost_round = 5000, early_stopping_rounds = 40, valid_sets = [train_set, val_set], verbose_eval = 1000)
        y_pred = model.predict(t[features])
        t['demand'] = y_pred
        #feature_importances[i] = model.feature_importance()
        #i+=1
        
        TEST = pd.concat([TEST,t],axis=0)
        del x_v, y_v, t, train_set, val_set, y_pred, model
        gc.collect()

'''
feature_importances['average'] = feature_importances[[i for i in range()]].mean(axis=1)
feature_importances.to_csv('lgb_feature_importances.csv')

plt.figure(figsize=(16, 12))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(20), x='average', y='feature');
plt.title('20 TOP feature importance over {} folds average'.format(folds.n_splits));
'''

In [17]:
predictions1 = TEST[TEST['id'].apply(lambda x: "validation" in x)][['id', 'date', 'demand']]
predictions2 = TEST[TEST['id'].apply(lambda x: "evaluation" in x)][['id', 'date', 'demand']]

In [18]:
prediction1 = pd.pivot(predictions1, index = 'id', columns = 'date', values = 'demand').reset_index()
prediction1.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [19]:
prediction2 = pd.pivot(predictions2, index = 'id', columns = 'date', values = 'demand').reset_index()
prediction2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [20]:
predictions = pd.concat([prediction1,prediction2],axis=0)
predictions.to_csv("lgbm_submission.csv",index=False)